In [2]:
# 필요 라이브러리

# 분석을 위한 라이브러리
import pandas as pd

# 에러메세지 안나오게
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns',None)

import plotly.graph_objects as go
import plotly.express as px

# 그래프 한글표시
import matplotlib
matplotlib.rcParams['font.family'] ='Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] =False


# 디비 연결
import psycopg2
import json

def rds_python_conn():
    # rds와 python 연동을 위한 정보 입력
    with open('postgres.info.json','r') as json_file:
        db_info = json.load(json_file)

    host = db_info.get('host')
    dbname = 'postgres' 
    user = 'postgres' 
    password = db_info.get('password')
    port = 5432

    connection = psycopg2.connect(host=host,
                                  dbname=dbname ,
                                  user=user ,
                                  password=password ,
                                  port=port)
    return connection

# 위도경도 계산시 필요 라이브러리
from haversine import haversine

### 판매자 정보

In [23]:
SQL = """
select A.mql_id,A.seller_id,seller_zip_code_prefix,seller_city,seller_state,온오프라인
from (SELECT m.mql_id,seller_id,business_segment,lead_type,lead_behaviour_profile,business_type,
       CASE
           WHEN lead_type IN ('industry','offline') THEN '오프라인'
           WHEN lead_type IN ('online_medium', 'online_big', 'online_top', 'online_small', 'online_beginner', 'other_publicities') THEN '온라인'
           ELSE 'other'
       END AS "온오프라인"
FROM mql m
JOIN closed_deals c ON m.mql_id = c.mql_id
WHERE seller_id in (SELECT seller_id FROM order_items)) A join sellers s on A.seller_id=s.seller_id                                                                           
;
"""

mql_closed = pd.read_sql(SQL, rds_python_conn())
mql_closed

,mql_id,seller_id,seller_zip_code_prefix,seller_city,seller_state,온오프라인
0,3690,2187,30421,belo horizonte,MG,온라인
1,1076,3397,93542,novo hamburgo,RS,온라인
2,1265,3242,11015,santos,SP,오프라인
3,2407,3046,30668,belo horizonte,MG,온라인
4,5147,2829,92010,canoas,RS,오프라인
...,...,...,...,...,...,...
375,7243,511,7194,guarulhos,SP,온라인
376,4597,1811,17400,garca,SP,온라인
377,615,3001,29107,vila velha,ES,온라인
378,3092,1496,8275,sao paulo,SP,온라인


In [26]:
SQL = """
select A.mql_id,A.seller_id,seller_zip_code_prefix,seller_city,seller_state,온오프라인
from (SELECT m.mql_id,seller_id,business_segment,lead_type,lead_behaviour_profile,business_type,
       CASE
           WHEN lead_type IN ('industry','offline') THEN '오프라인'
           WHEN lead_type IN ('online_medium', 'online_big', 'online_top', 'online_small', 'online_beginner', 'other_publicities') THEN '온라인'
           ELSE 'other'
       END AS "온오프라인"
FROM mql m
JOIN closed_deals c ON m.mql_id = c.mql_id
WHERE seller_id in (SELECT seller_id FROM order_items)) A join sellers s on A.seller_id=s.seller_id                                                                           
;
"""

SQL2 = """
SELECT geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_state
FROM geoloc
groupby 
;
"""

mql_closed = pd.read_sql(SQL, rds_python_conn())

geoloc = pd.read_sql(SQL2, rds_python_conn())
geoloc  = geoloc.drop_duplicates(subset=['geolocation_zip_code_prefix','geolocation_state'])

em = mql_closed.merge(geoloc, left_on='seller_zip_code_prefix',right_on='geolocation_zip_code_prefix')
em = em[['mql_id','seller_id','seller_zip_code_prefix','seller_state','온오프라인','geolocation_lat','geolocation_lng']]
em

# 판매기록이 있는 판매자들

,mql_id,seller_id,seller_zip_code_prefix,seller_state,온오프라인,geolocation_lat,geolocation_lng
0,3690,2187,30421,MG,온라인,-19.939689,-43.975338
1,5608,137,30421,MG,온라인,-19.939689,-43.975338
2,1076,3397,93542,RS,온라인,-29.688181,-51.091561
3,1265,3242,11015,SP,오프라인,-23.959748,-46.313592
4,2407,3046,30668,MG,온라인,-20.009133,-44.029840
...,...,...,...,...,...,...,...
374,795,2884,72318,DF,온라인,-15.863133,-48.085951
375,7243,511,7194,SP,온라인,-23.444149,-46.506552
376,615,3001,29107,ES,온라인,-20.348450,-40.298021
377,3092,1496,8275,SP,온라인,-23.577140,-46.482762


### customer 정보

In [27]:
SQL = """
SELECT order_id,o.customer_id,customer_unique_id,customer_zip_code_prefix,customer_state
FROM customers c 
    join "order" o on o.customer_id = c.customer_id
where o.order_status in ('delivered','shipped','invoiced','processing')
;
"""
customers = pd.read_sql(SQL, rds_python_conn())

SQL = """
SELECT *
FROM geoloc  
;
"""

geoloc = pd.read_sql(SQL, rds_python_conn())
geoloc  = geoloc.drop_duplicates(subset=['geolocation_zip_code_prefix','geolocation_state'])


SQL = """
SELECT order_id,seller_id
FROM order_items  
;
"""

order_items = pd.read_sql(SQL, rds_python_conn())



customers_tmp = customers.merge(order_items,on='order_id',how='left')
tmp = customers_tmp.merge(geoloc,left_on='customer_zip_code_prefix',right_on='geolocation_zip_code_prefix')
tmp = tmp[['customer_unique_id','seller_id','geolocation_zip_code_prefix','geolocation_lat','geolocation_lng','geolocation_state']]
tmp = tmp.drop_duplicates().reset_index(drop=True)
tmp

,customer_unique_id,seller_id,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_state
0,39184,2940.0,3149,-23.574809,-46.587471,SP
1,37325,3008.0,3149,-23.574809,-46.587471,SP
2,39184,2327.0,3149,-23.574809,-46.587471,SP
3,71798,3268.0,47813,-12.169860,-44.988369,BA
4,7710,2236.0,47813,-12.169860,-44.988369,BA
...,...,...,...,...,...,...
98428,17676,3208.0,25932,-22.597556,-43.176153,RJ
98429,38084,2509.0,77829,-7.222311,-48.242792,TO
98430,11584,1824.0,83870,-26.001234,-49.668993,PR
98431,3043,2594.0,5127,-23.495994,-46.742453,SP


### 판매자와 구매자간의 거리 계산(위도,경도)

In [28]:
em_online = em[em.온오프라인=='온라인'].reset_index(drop=True)   # 온라인채널 판매자정보
em_offline = em[em.온오프라인=='오프라인'].reset_index(drop=True)  # 오프라인채널 판매자정보

tmp_notfree = tmp[tmp['seller_id'].isin(em_offline.seller_id)].reset_index(drop=True) # 오프라인채널 판매자의 제품을 산 고객정보
tmp_free = tmp[tmp['seller_id'].isin(em_online.seller_id)].reset_index(drop=True)     # 온라인채널 판매자의 제품을 산 고객정보

em_online_list = []   # 온라인채널 판매자와 고객간의 거리 리스트
em_offline_list = []   # 오프라인채널 판매자와 고객간의 거리 리스트

for i in range(len(em_online)):
    seller = (em.loc[i,'geolocation_lat'],em.loc[i,'geolocation_lng'])
    if em_online['seller_id'][i] in tmp_free['seller_id'].unique():
        seller_info = tmp_free[tmp_free['seller_id'] == em_online['seller_id'][i]]  # 해당 판매자의 정보 추출
        for j in range(len(seller_info)):
            geolocation_lat = seller_info.iloc[j]['geolocation_lat']
            geolocation_lng = seller_info.iloc[j]['geolocation_lng']
            customer = (geolocation_lat,geolocation_lng)
            q = haversine(seller, customer, unit = 'km')
            em_online_list.append(q)
    else:
        pass

for i in range(len(em_offline)):
    seller = (em.loc[i,'geolocation_lat'],em.loc[i,'geolocation_lng'])
    if em_offline['seller_id'][i] in tmp_notfree['seller_id'].unique():
        seller_info = tmp_notfree[tmp_notfree['seller_id'] == em_offline['seller_id'][i]]  # 해당 판매자의 정보 추출
        for j in range(len(seller_info)):
            geolocation_lat = seller_info.iloc[j]['geolocation_lat']
            geolocation_lng = seller_info.iloc[j]['geolocation_lng']
            customer = (geolocation_lat,geolocation_lng)
            q = haversine(seller, customer, unit = 'km')
            em_offline_list.append(q)
    else:
        pass

[386.18025328790776, 422.5904611011318, 357.3063763040481, 1334.585470650588, 771.6355641535506, 805.1578858829458, 1084.682822985606, 848.1645605929806, 1063.5879021937076, 3.582944666962422, 3.828278439997978, 1083.6555559356873, 2938.8781720455604, 929.3328518996468, 1230.9490582691121, 260.2611626580212, 813.2417523762192, 340.7106077146745, 960.2450045522266, 1571.8396612659037, 855.0493130968136, 706.9339309357288, 1519.0404374291302, 1519.0404374291302, 884.8357557764625, 37.41981171077295, 808.1206994472672, 1073.7693290770644, 976.1341043263478, 827.0987753211652, 852.1178053142222, 808.8543405649248, 329.108377237509, 53.82495412978725, 812.266337979392, 111.40778163110208, 262.4499232883248, 2186.5475070505354, 3154.765120251894, 1150.033246965204, 875.2307177105108, 1059.3906651715638, 1101.486049059754, 806.0517481346272, 1307.4160706307364, 502.72970641478975, 40.33102765519901, 863.9959721151118, 1079.9554656520327, 3032.1653243565597, 1299.382487052978, 808.212032892049

In [57]:
tmp = [sum(em_online_list)/len(em_online_list),sum(em_offline_list)/len(em_offline_list)]
x_line = ['온라인','오프라인']

fig = px.bar(x=x_line, y=tmp, title='온오프라인별 판매자와 구매자의 평균 거리')
fig.update_traces(marker_color=['#A0BFE0','#4A55A2'])
fig.update_layout(width=600)
# x축 이름 설정
fig.update_xaxes(title_text='온/오프라인')
# y축 이름 설정
fig.update_yaxes(title_text='평균 거리 (km)')

for i, value in enumerate(tmp):
    color = 'black' if value < sum(tmp) / len(tmp) else 'white'
    fig.add_annotation(
        x=x_line[i],
        y=value-20,
        text=f'{value:.2f} km',  # 글씨 내용 및 포맷
        showarrow=False,
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor='gray',
        ax=0,
        ay=-50,  # 글씨를 더 아래로 내림
        font=dict(color=color)   # 글씨 색상을 빨간색으로 변경
    )

fig.show()